In [3]:
import sys
sys.path.append("..")

from helpers.load_data import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [13]:
dfs = load_datafolder('processed')
df = dfs['final_data_surrey.csv']

/home/tanny/vanml-hacks-2023/notebooks/../helpers/load_data.py:20: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_filepath, parse_dates=[1, 2])
/home/tanny/vanml-hacks-2023/notebooks/../helpers/load_data.py:20: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_filepath, parse_dates=[1, 2])
/home/tanny/vanml-hacks-2023/notebooks/../helpers/load_data.py:20: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_filepath, parse_dates=[1, 2])


In [15]:
features = ['DateStart','CallerDemographicsGender', 'CallType', 'CallerDemographicsCallerType', 'ContactMethod']

encoder = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown="ignore"), features)
    ],
    remainder='passthrough'  # Keep numeric features that are not transformed
)


lr_pipeline = make_pipeline(
    StandardScaler(with_mean=False),
    RandomForestRegressor()
)

X = df[features] 
y = df["CallLength"]
x_transform = encoder.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(x_transform, y, test_size=.2, random_state=42)

fotted = lr_pipeline.fit(X_train, y_train)

fotted.score(X_test, y_test)

/home/morris/Documents/coding/vanML-2023/notebooks/../helpers/load_data.py:20: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_filepath)
/home/morris/Documents/coding/vanML-2023/notebooks/../helpers/load_data.py:20: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_filepath)
/home/morris/Documents/coding/vanML-2023/notebooks/../helpers/load_data.py:20: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_filepath)


In [6]:
def train_model_on_city(city_name: str):
    features = ['DateStart','CallerDemographicsGender', 'CallType', 'CallerDemographicsCallerType', 'ContactMethod']
    df = dfs[f'final_data_{city_name}.csv']

    encoder = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown="ignore"), features)
        ],
        remainder='passthrough'  # Keep numeric features that are not transformed
    )
    X = df[features] 
    y = df["CallLength"]
    X_transform = encoder.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X_transform, y, test_size=0.2, random_state=42)
    
    rf_pipeline = make_pipeline(
        StandardScaler(with_mean=False),
        RandomForestRegressor()
    )   

    return rf_pipeline.fit(X_train, y_train), X_test, y_test

In [14]:
cities = ['burnaby', 'nw', 'surrey', 'vancouver', 'victoria']

for city in cities:
    rf_model, X_test, y_test = train_model_on_city(city)
    print(f'{city} has a score of {rf_model.score(X_test, y_test)}')

burnaby has a score of 0.43967508912278297
nw has a score of 0.44684585575247404
surrey has a score of 0.307604377586296
